In [ ]:
import pandas as pd
import numpy as np
import re
import altair as alt
pd.set_option('display.max_columns', None)

In [ ]:
!pip install geopandas
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Sept. 2020 ride data

In [ ]:
from zipfile import ZipFile
import requests
import io

# Here we are getting the zip file
citibikeZipFile = requests.get('https://s3.amazonaws.com/tripdata/202009-citibike-tripdata.csv.zip')

# Here we are uncompressing it and reading the whole package into a variable
zipFiles = ZipFile(io.BytesIO(citibikeZipFile.content))

# And here we are looping through the package and printing the names of the files
for name in zipFiles.namelist():
  print(name)

# Finally, we can take that loaded uncopressed package and use it to just load one file to the Pandas dataframe:
sept2020 = pd.read_csv(zipFiles.open('202009-citibike-tripdata.csv'))

202009-citibike-tripdata.csv


In [ ]:
sept2020.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,4225,2020-09-01 00:00:01.0430,2020-09-01 01:10:26.6350,3508,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,116,W 17 St & 8 Ave,40.741776,-74.001497,44317,Customer,1979,1
1,1868,2020-09-01 00:00:04.8320,2020-09-01 00:31:13.7650,3621,27 Ave & 9 St,40.773983,-73.930913,3094,Graham Ave & Withers St,40.716981,-73.944859,37793,Customer,1991,1
2,1097,2020-09-01 00:00:06.8990,2020-09-01 00:18:24.2260,3492,E 118 St & Park Ave,40.800539,-73.941995,3959,Edgecombe Ave & W 145 St,40.823498,-73.943860,41438,Subscriber,1984,1
3,1473,2020-09-01 00:00:07.7440,2020-09-01 00:24:41.1800,3946,St Nicholas Ave & W 137 St,40.818477,-73.947568,4002,W 144 St & Adam Clayton Powell Blvd,40.820877,-73.939249,35860,Customer,1990,2
4,1193,2020-09-01 00:00:12.2020,2020-09-01 00:20:05.5470,3081,Graham Ave & Grand St,40.711863,-73.944024,3048,Putnam Ave & Nostrand Ave,40.684020,-73.949770,26396,Customer,1969,0


## Cleaning the data

In [ ]:
sept2020.columns = sept2020.columns.str.lower()

In [ ]:
sept2020.columns = sept2020.columns.str.replace(" ", "_")

In [ ]:
sept2020

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,4225,2020-09-01 00:00:01.0430,2020-09-01 01:10:26.6350,3508,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,116,W 17 St & 8 Ave,40.741776,-74.001497,44317,Customer,1979,1
1,1868,2020-09-01 00:00:04.8320,2020-09-01 00:31:13.7650,3621,27 Ave & 9 St,40.773983,-73.930913,3094,Graham Ave & Withers St,40.716981,-73.944859,37793,Customer,1991,1
2,1097,2020-09-01 00:00:06.8990,2020-09-01 00:18:24.2260,3492,E 118 St & Park Ave,40.800539,-73.941995,3959,Edgecombe Ave & W 145 St,40.823498,-73.943860,41438,Subscriber,1984,1
3,1473,2020-09-01 00:00:07.7440,2020-09-01 00:24:41.1800,3946,St Nicholas Ave & W 137 St,40.818477,-73.947568,4002,W 144 St & Adam Clayton Powell Blvd,40.820877,-73.939249,35860,Customer,1990,2
4,1193,2020-09-01 00:00:12.2020,2020-09-01 00:20:05.5470,3081,Graham Ave & Grand St,40.711863,-73.944024,3048,Putnam Ave & Nostrand Ave,40.684020,-73.949770,26396,Customer,1969,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488220,1818,2020-09-30 23:59:48.8460,2020-10-01 00:30:07.7810,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,3836,Bushwick Ave & Linden St,40.691460,-73.921460,36220,Subscriber,1993,1
2488221,1553,2020-09-30 23:59:53.7240,2020-10-01 00:25:47.1760,390,Duffield St & Willoughby St,40.692216,-73.984284,3799,6 Ave & W 34 St,40.749640,-73.988050,31092,Subscriber,1995,1
2488222,626,2020-09-30 23:59:54.6670,2020-10-01 00:10:21.6050,3901,Stanhope St & Fairview Ave,40.709980,-73.911160,3083,Bushwick Ave & Powers St,40.712477,-73.941000,47224,Subscriber,1954,2
2488223,325,2020-09-30 23:59:56.2120,2020-10-01 00:05:21.3060,3148,E 84 St & 1 Ave,40.775655,-73.950686,3338,2 Ave & E 99 St,40.786259,-73.945526,16759,Subscriber,1995,2


## Total number of trips

In [ ]:
sept2020.count()

tripduration               2488225
starttime                  2488225
stoptime                   2488225
start_station_id           2488225
start_station_name         2488225
start_station_latitude     2488225
start_station_longitude    2488225
end_station_id             2488225
end_station_name           2488225
end_station_latitude       2488225
end_station_longitude      2488225
bikeid                     2488225
usertype                   2488225
birth_year                 2488225
gender                     2488225
dtype: int64

###### Total number of trips was 2.489 million

## Top 5 stations by number of trip starts

In [ ]:
sept2020.groupby('start_station_name').starttime.count().sort_values(ascending=False).head(5)

start_station_name
W 21 St & 6 Ave          13875
West St & Chambers St    13225
12 Ave & W 40 St         12869
1 Ave & E 68 St          12816
E 17 St & Broadway       11264
Name: starttime, dtype: int64

## Top 5 stations by number of trip stops

In [ ]:
sept2020.groupby('end_station_name').stoptime.count().sort_values(ascending=False).head(5)

end_station_name
W 21 St & 6 Ave          13967
West St & Chambers St    13783
12 Ave & W 40 St         12922
1 Ave & E 68 St          12689
E 17 St & Broadway       11446
Name: stoptime, dtype: int64

## Median number of starts per station

In [ ]:
sept2020.groupby('start_station_name').starttime.count().median()

1457.0

## Median number of stops per station

In [ ]:
sept2020.groupby('end_station_name').stoptime.count().median()

1434.5

# Sept. 2021 ride data

In [ ]:
# Here we are getting the zip file
citibikeZipFile = requests.get('https://s3.amazonaws.com/tripdata/202109-citibike-tripdata.csv.zip')

# Here we are uncompressing it and reading the whole package into a variable
zipFiles = ZipFile(io.BytesIO(citibikeZipFile.content))

# And here we are looping through the package and printing the names of the files
for name in zipFiles.namelist():
  print(name)

# Finally, we can take that loaded uncopressed package and use it to just load one file to the Pandas dataframe:
sept2021 = pd.read_csv(zipFiles.open('202109-citibike-tripdata.csv'))

202109-citibike-tripdata.csv
__MACOSX/._202109-citibike-tripdata.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
sept2021

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A789BD2808767C24,classic_bike,2021-09-16 12:34:39,2021-09-16 12:36:05,Clinton St\t& Cherry St,5190.09,Madison St & Clinton St,5190.07,40.711471,-73.986726,40.712690,-73.987763,member
1,B3E45CE415FDF1E7,classic_bike,2021-09-28 20:36:25,2021-09-28 20:37:58,Clinton St & Grand St,5303.06,Madison St & Clinton St,5190.07,40.715595,-73.987030,40.712690,-73.987763,member
2,50433A2E1F35CBBB,classic_bike,2021-09-26 20:08:43,2021-09-26 20:25:36,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual
3,C478A0B3E61B504F,electric_bike,2021-09-06 18:25:20,2021-09-06 18:30:21,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual
4,2B82A80EECFC4458,electric_bike,2021-09-07 18:39:08,2021-09-07 18:54:03,E 65 St & 2 Ave,6860.12,28 St & 41 Ave,6462.19,40.764719,-73.962221,40.751047,-73.937970,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280555,720BBDEE43921F6D,classic_bike,2021-09-21 17:48:25,2021-09-21 18:08:46,E 11 St & 1 Ave,5746.14,Broadway & W 41 St,6560.01,40.729538,-73.984267,40.755136,-73.986580,member
3280556,4ED4E160CC1D1C65,classic_bike,2021-09-11 12:53:06,2021-09-11 13:08:40,3 Ave & E 72 St,7028.04,E 72 St & Park Ave,6998.08,40.769943,-73.960607,40.771183,-73.964094,member
3280557,E924ABF461609933,classic_bike,2021-09-18 13:36:50,2021-09-18 13:39:49,Broadway & W 36 St,6441.01,Broadway & W 41 St,6560.01,40.750977,-73.987654,40.755136,-73.986580,member
3280558,8A768D642836C58A,classic_bike,2021-09-10 23:36:49,2021-09-10 23:56:42,Grand St & Greene St,5500.02,Cherry St,5181.04,40.721700,-74.002381,40.712199,-73.979481,casual


## Cleaning the data

In [ ]:
sept2021.columns = sept2021.columns.str.lower()
sept2021.columns = sept2021.columns.str.replace(" ", "_")
sept2021

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A789BD2808767C24,classic_bike,2021-09-16 12:34:39,2021-09-16 12:36:05,Clinton St\t& Cherry St,5190.09,Madison St & Clinton St,5190.07,40.711471,-73.986726,40.712690,-73.987763,member
1,B3E45CE415FDF1E7,classic_bike,2021-09-28 20:36:25,2021-09-28 20:37:58,Clinton St & Grand St,5303.06,Madison St & Clinton St,5190.07,40.715595,-73.987030,40.712690,-73.987763,member
2,50433A2E1F35CBBB,classic_bike,2021-09-26 20:08:43,2021-09-26 20:25:36,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual
3,C478A0B3E61B504F,electric_bike,2021-09-06 18:25:20,2021-09-06 18:30:21,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual
4,2B82A80EECFC4458,electric_bike,2021-09-07 18:39:08,2021-09-07 18:54:03,E 65 St & 2 Ave,6860.12,28 St & 41 Ave,6462.19,40.764719,-73.962221,40.751047,-73.937970,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280555,720BBDEE43921F6D,classic_bike,2021-09-21 17:48:25,2021-09-21 18:08:46,E 11 St & 1 Ave,5746.14,Broadway & W 41 St,6560.01,40.729538,-73.984267,40.755136,-73.986580,member
3280556,4ED4E160CC1D1C65,classic_bike,2021-09-11 12:53:06,2021-09-11 13:08:40,3 Ave & E 72 St,7028.04,E 72 St & Park Ave,6998.08,40.769943,-73.960607,40.771183,-73.964094,member
3280557,E924ABF461609933,classic_bike,2021-09-18 13:36:50,2021-09-18 13:39:49,Broadway & W 36 St,6441.01,Broadway & W 41 St,6560.01,40.750977,-73.987654,40.755136,-73.986580,member
3280558,8A768D642836C58A,classic_bike,2021-09-10 23:36:49,2021-09-10 23:56:42,Grand St & Greene St,5500.02,Cherry St,5181.04,40.721700,-74.002381,40.712199,-73.979481,casual


## Total number of trips

In [ ]:
sept2021.count()

ride_id               3280560
rideable_type         3280560
started_at            3280560
ended_at              3280560
start_station_name    3280542
start_station_id      3280542
end_station_name      3264876
end_station_id        3264876
start_lat             3280560
start_lng             3280560
end_lat               3271443
end_lng               3271443
member_casual         3280560
dtype: int64

###### The total number of trips was 3.28 million.

## Top 5 stations by number of trip starts

In [ ]:
sept2021.groupby('start_station_name').started_at.count().sort_values(ascending=False).head(5)

start_station_name
W 21 St & 6 Ave             14435
E 17 St & Broadway          14065
Broadway & E 14 St          13155
Cleveland Pl & Spring St    12987
W 20 St & 10 Ave            12918
Name: started_at, dtype: int64

## Top 5 stations by number of trip stops

In [ ]:
sept2021.groupby('start_station_name').ended_at.count().sort_values(ascending=False).head(5)

start_station_name
W 21 St & 6 Ave             14435
E 17 St & Broadway          14065
Broadway & E 14 St          13155
Cleveland Pl & Spring St    12987
W 20 St & 10 Ave            12918
Name: ended_at, dtype: int64

## Median number of starts per station

In [ ]:
sept2021.groupby('start_station_name').started_at.count().median()

1133.5

## Median number of stops per station

In [ ]:
sept2021.groupby('end_station_name').ended_at.count().median()

1078.0

# Mapping

In [ ]:
ntaData = gpd.read_file('https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Neighborhood_Tabulation_Areas_2020/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson')
ntaData

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape__Area,Shape__Length,geometry
0,1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),3.532193e+07,28914.314717,"POLYGON ((-73.93214 40.72817, -73.93253 40.727..."
1,2,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),2.886227e+07,28155.614512,"POLYGON ((-73.95814 40.72441, -73.95772 40.724..."
2,3,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),1.520934e+07,18252.346181,"POLYGON ((-73.95024 40.70548, -73.94984 40.705..."
3,4,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),5.226632e+07,43171.264181,"POLYGON ((-73.92406 40.71412, -73.92404 40.714..."
4,5,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,9.985438e+06,14337.853843,"POLYGON ((-73.99237 40.68970, -73.99436 40.690..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,258,5,Staten Island,085,SI0391,Freshkills Park (South),FrshklPK_S,9,SI03,SI03 South Shore (CD 3 Approximation),4.775867e+07,33945.050028,"POLYGON ((-74.20059 40.57952, -74.19888 40.579..."
258,259,5,Staten Island,085,SI9561,Fort Wadsworth,FtWdswrth,6,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,9.883809e+06,14852.484590,"POLYGON ((-74.05975 40.59386, -74.06014 40.594..."
259,260,5,Staten Island,085,SI9591,Hoffman & Swinburne Islands,HffmnIsl,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,6.357020e+05,4743.128127,"MULTIPOLYGON (((-74.05314 40.57771, -74.05406 ..."
260,261,5,Staten Island,085,SI9592,Miller Field,MllrFld,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,1.086771e+07,19196.371774,"POLYGON ((-74.08469 40.57149, -74.08595 40.570..."


## Preparing your 2020 data for mapping

In [ ]:
#2020 starts


sept2020stationsStarts = sept2020.groupby(['start_station_name']).agg({'start_station_latitude':'min','start_station_longitude':'min', 'tripduration':'count'}).reset_index()
sept2020stationsStarts.rename(columns={'tripduration':'tripCount'},inplace=True)
sept2020stationsStarts = gpd.GeoDataFrame(data=sept2020stationsStarts, geometry=gpd.points_from_xy(x=sept2020stationsStarts['start_station_longitude'], y=sept2020stationsStarts['start_station_latitude']), crs='epsg:4326')
sept2020stationsStarts.head()

,start_station_name,start_station_latitude,start_station_longitude,tripCount,geometry
0,1 Ave & E 110 St,40.792327,-73.938300,2161,POINT (-73.93830 40.79233)
1,1 Ave & E 16 St,40.732219,-73.981656,7685,POINT (-73.98166 40.73222)
2,1 Ave & E 18 St,40.733812,-73.980544,7611,POINT (-73.98054 40.73381)
3,1 Ave & E 30 St,40.741444,-73.975361,4818,POINT (-73.97536 40.74144)
4,1 Ave & E 39 St,40.747140,-73.971130,6349,POINT (-73.97113 40.74714)


In [ ]:
#2020 stops


sept2020stationsStops = sept2020.groupby(['end_station_name']).agg({'end_station_latitude':'min','end_station_longitude':'min', 'tripduration':'count'}).reset_index()
sept2020stationsStops.rename(columns={'tripduration':'tripCount'},inplace=True)
sept2020stationsStops = gpd.GeoDataFrame(data=sept2020stationsStops, geometry=gpd.points_from_xy(x=sept2020stationsStops['end_station_longitude'], y=sept2020stationsStops['end_station_latitude']), crs='epsg:4326')
sept2020stationsStops.head()

,end_station_name,end_station_latitude,end_station_longitude,tripCount,geometry
0,1 Ave & E 110 St,40.792327,-73.938300,2177,POINT (-73.93830 40.79233)
1,1 Ave & E 16 St,40.732219,-73.981656,7546,POINT (-73.98166 40.73222)
2,1 Ave & E 18 St,40.733812,-73.980544,7503,POINT (-73.98054 40.73381)
3,1 Ave & E 30 St,40.741444,-73.975361,4840,POINT (-73.97536 40.74144)
4,1 Ave & E 39 St,40.747140,-73.971130,6374,POINT (-73.97113 40.74714)


In [ ]:
sept2020stationsStops.tripCount.sort_values(ascending=False)

999     13967
1080    13783
22      12922
8       12689
434     11446
        ...  
766         1
753         1
637         1
877         1
605         1
Name: tripCount, Length: 1112, dtype: int64

## Preparing 2021 data for mapping

### Create new column 'tripCount'

In [ ]:
sept2021['tripCount'] = 1
sept2021

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,tripCount
0,A789BD2808767C24,classic_bike,2021-09-16 12:34:39,2021-09-16 12:36:05,Clinton St\t& Cherry St,5190.09,Madison St & Clinton St,5190.07,40.711471,-73.986726,40.712690,-73.987763,member,1
1,B3E45CE415FDF1E7,classic_bike,2021-09-28 20:36:25,2021-09-28 20:37:58,Clinton St & Grand St,5303.06,Madison St & Clinton St,5190.07,40.715595,-73.987030,40.712690,-73.987763,member,1
2,50433A2E1F35CBBB,classic_bike,2021-09-26 20:08:43,2021-09-26 20:25:36,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual,1
3,C478A0B3E61B504F,electric_bike,2021-09-06 18:25:20,2021-09-06 18:30:21,LaGuardia Pl & W 3 St,5721.14,Greenwich Ave & Charles St,5914.08,40.729170,-73.998102,40.735238,-74.000271,casual,1
4,2B82A80EECFC4458,electric_bike,2021-09-07 18:39:08,2021-09-07 18:54:03,E 65 St & 2 Ave,6860.12,28 St & 41 Ave,6462.19,40.764719,-73.962221,40.751047,-73.937970,casual,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280555,720BBDEE43921F6D,classic_bike,2021-09-21 17:48:25,2021-09-21 18:08:46,E 11 St & 1 Ave,5746.14,Broadway & W 41 St,6560.01,40.729538,-73.984267,40.755136,-73.986580,member,1
3280556,4ED4E160CC1D1C65,classic_bike,2021-09-11 12:53:06,2021-09-11 13:08:40,3 Ave & E 72 St,7028.04,E 72 St & Park Ave,6998.08,40.769943,-73.960607,40.771183,-73.964094,member,1
3280557,E924ABF461609933,classic_bike,2021-09-18 13:36:50,2021-09-18 13:39:49,Broadway & W 36 St,6441.01,Broadway & W 41 St,6560.01,40.750977,-73.987654,40.755136,-73.986580,member,1
3280558,8A768D642836C58A,classic_bike,2021-09-10 23:36:49,2021-09-10 23:56:42,Grand St & Greene St,5500.02,Cherry St,5181.04,40.721700,-74.002381,40.712199,-73.979481,casual,1


In [ ]:
#2021 starts


sept2021stationsStarts = sept2021.groupby(['start_station_name']).agg({'start_lat':'min','start_lng':'min', 'tripCount':'count'}).reset_index()
sept2021stationsStarts = gpd.GeoDataFrame(data=sept2021stationsStarts, geometry=gpd.points_from_xy(x=sept2021stationsStarts['start_lng'], y=sept2021stationsStarts['start_lat']), crs='epsg:4326')
sept2021stationsStarts.head()

,start_station_name,start_lat,start_lng,tripCount,geometry
0,1 Ave & E 110 St,40.792327,-73.938300,2376,POINT (-73.93830 40.79233)
1,1 Ave & E 16 St,40.732219,-73.981656,9873,POINT (-73.98166 40.73222)
2,1 Ave & E 18 St,40.733812,-73.980544,8279,POINT (-73.98054 40.73381)
3,1 Ave & E 30 St,40.741444,-73.975361,4671,POINT (-73.97536 40.74144)
4,1 Ave & E 39 St,40.747140,-73.971130,7507,POINT (-73.97113 40.74714)


In [ ]:
#2021 stops


sept2021stationsStops = sept2021.groupby(['end_station_name']).agg({'end_lat':'min','end_lng':'min', 'tripCount':'count'}).reset_index()
sept2021stationsStops = gpd.GeoDataFrame(data=sept2021stationsStops, geometry=gpd.points_from_xy(x=sept2021stationsStops['end_lng'], y=sept2021stationsStops['end_lat']), crs='epsg:4326')
sept2021stationsStops.head()

,end_station_name,end_lat,end_lng,tripCount,geometry
0,1 Ave & E 110 St,40.792327,-73.938300,2445,POINT (-73.93830 40.79233)
1,1 Ave & E 16 St,40.732219,-73.981656,9794,POINT (-73.98166 40.73222)
2,1 Ave & E 18 St,40.733812,-73.980544,8175,POINT (-73.98054 40.73381)
3,1 Ave & E 30 St,40.741444,-73.975361,4663,POINT (-73.97536 40.74144)
4,1 Ave & E 39 St,40.747140,-73.971130,7553,POINT (-73.97113 40.74714)


## Producing GEOJSON file

In [ ]:
sept2020stationsStarts.to_file('sept2020Starts.geojson', driver='GeoJSON')
sept2020stationsStops.to_file('sept2020Stops.geojson', driver='GeoJSON')
sept2021stationsStarts.to_file('sept2021Starts.geojson', driver='GeoJSON')
sept2021stationsStops.to_file('sept2021Stops.geojson', driver='GeoJSON')